<a href="https://colab.research.google.com/github/msrjdin/TransformerUseCases/blob/master/Text%20Classification/camembert-base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip3 install transformers

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('Data.csv')

In [9]:
df.dropna(inplace=True)

In [10]:
df.shape

(23853, 5)

In [11]:
df.head()

,asin,Rating5,title,body1,price
0,B0009N5L7K,0,Stupid phone,BUY SERVICE,49.95
1,B0009N5L7K,0,Exellent Service,nextel nearly year started time last year Moto...,49.95
2,B0009N5L7K,1,I love it,"got say easy use, hear person talking fine pro...",49.95
3,B0009N5L7K,0,Phones locked,1 star phones locked pay additional fees unlock,49.95
4,B0009N5L7K,1,Excellent product,product good. used cell phone one projects wor...,49.95


In [12]:
df.rename(columns = {'body1':'text', 'Rating5':'5star'}, inplace=True)

In [13]:
df.reset_index(drop= True, inplace=True)

In [14]:
df.head()

,asin,5star,title,text,price
0,B0009N5L7K,0,Stupid phone,BUY SERVICE,49.95
1,B0009N5L7K,0,Exellent Service,nextel nearly year started time last year Moto...,49.95
2,B0009N5L7K,1,I love it,"got say easy use, hear person talking fine pro...",49.95
3,B0009N5L7K,0,Phones locked,1 star phones locked pay additional fees unlock,49.95
4,B0009N5L7K,1,Excellent product,product good. used cell phone one projects wor...,49.95


In [15]:
from sklearn.model_selection import train_test_split
X= df['text'].copy()
y= df['5star']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [17]:
#Setting the model_name runs the model required
model_name = "camembert-base"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

In [18]:
tr_enc = tokenizer(list(X_train.values), padding=True, truncation=True, max_length=100)
tst_enc = tokenizer(list(X_test.values),  padding=True, truncation=True,  max_length=100)

In [19]:
import torch

class buildingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = buildingDataset(tr_enc, list(y_train.values))
val_dataset = buildingDataset(tst_enc, list(y_test.values))

In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

trainer = Trainer(
    model=pt_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_metrics
)


In [22]:
trainer.train()

{'loss': 0.693388557434082, 'learning_rate': 5e-06, 'epoch': 0.009578544061302681, 'step': 10}
{'loss': 0.6937730789184571, 'learning_rate': 1e-05, 'epoch': 0.019157088122605363, 'step': 20}
{'loss': 0.6921953201293946, 'learning_rate': 1.5e-05, 'epoch': 0.028735632183908046, 'step': 30}
{'loss': 0.682624626159668, 'learning_rate': 2e-05, 'epoch': 0.038314176245210725, 'step': 40}
{'loss': 0.6828376770019531, 'learning_rate': 2.5e-05, 'epoch': 0.04789272030651341, 'step': 50}
{'loss': 0.6575931549072266, 'learning_rate': 3e-05, 'epoch': 0.05747126436781609, 'step': 60}
{'loss': 0.6668800354003906, 'learning_rate': 3.5e-05, 'epoch': 0.06704980842911877, 'step': 70}
{'loss': 0.6121574401855469, 'learning_rate': 4e-05, 'epoch': 0.07662835249042145, 'step': 80}
{'loss': 0.5587615966796875, 'learning_rate': 4.5e-05, 'epoch': 0.08620689655172414, 'step': 90}
{'loss': 0.57872314453125, 'learning_rate': 5e-05, 'epoch': 0.09578544061302682, 'step': 100}
{'loss': 0.6299209594726562, 'learning_ra

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 0.4633270263671875, 'learning_rate': 3.968812877263582e-05, 'epoch': 0.4885057471264368, 'step': 510}
{'loss': 0.525897216796875, 'learning_rate': 3.943661971830986e-05, 'epoch': 0.49808429118773945, 'step': 520}
{'loss': 0.4208221435546875, 'learning_rate': 3.91851106639839e-05, 'epoch': 0.5076628352490421, 'step': 530}
{'loss': 0.4717376708984375, 'learning_rate': 3.8933601609657946e-05, 'epoch': 0.5172413793103449, 'step': 540}
{'loss': 0.415972900390625, 'learning_rate': 3.8682092555332e-05, 'epoch': 0.5268199233716475, 'step': 550}
{'loss': 0.4146331787109375, 'learning_rate': 3.843058350100604e-05, 'epoch': 0.5363984674329502, 'step': 560}
{'loss': 0.474444580078125, 'learning_rate': 3.817907444668008e-05, 'epoch': 0.5459770114942529, 'step': 570}
{'loss': 0.464599609375, 'learning_rate': 3.792756539235413e-05, 'epoch': 0.5555555555555556, 'step': 580}
{'loss': 0.5824127197265625, 'learning_rate': 3.767605633802817e-05, 'epoch': 0.5651340996168582, 'step': 590}
{'loss': 

{'loss': 0.39578857421875, 'learning_rate': 2.6106639839034207e-05, 'epoch': 1.0057471264367817, 'step': 1050}
{'loss': 0.3864501953125, 'learning_rate': 2.5855130784708247e-05, 'epoch': 1.0153256704980842, 'step': 1060}
{'loss': 0.372650146484375, 'learning_rate': 2.5603621730382295e-05, 'epoch': 1.024904214559387, 'step': 1070}
{'loss': 0.37137451171875, 'learning_rate': 2.535211267605634e-05, 'epoch': 1.0344827586206897, 'step': 1080}
{'loss': 0.4374755859375, 'learning_rate': 2.5100603621730386e-05, 'epoch': 1.0440613026819923, 'step': 1090}
{'loss': 0.371270751953125, 'learning_rate': 2.4849094567404426e-05, 'epoch': 1.053639846743295, 'step': 1100}
{'loss': 0.4509521484375, 'learning_rate': 2.4597585513078473e-05, 'epoch': 1.0632183908045978, 'step': 1110}
{'loss': 0.38736572265625, 'learning_rate': 2.4346076458752517e-05, 'epoch': 1.0727969348659003, 'step': 1120}
{'loss': 0.331951904296875, 'learning_rate': 2.409456740442656e-05, 'epoch': 1.082375478927203, 'step': 1130}
{'loss

TrainOutput(global_step=2088, training_loss=0.4453107753475964)

In [24]:
trainer.evaluate()


{'eval_loss': 0.4220174372596005, 'eval_accuracy': 0.8235047512576859, 'eval_f1': 0.8351820435860631, 'eval_precision': 0.8022060666833793, 'eval_recall': 0.8709853021230267, 'epoch': 2.0, 'step': 2088}


{'epoch': 2.0,
 'eval_accuracy': 0.8235047512576859,
 'eval_f1': 0.8351820435860631,
 'eval_loss': 0.4220174372596005,
 'eval_precision': 0.8022060666833793,
 'eval_recall': 0.8709853021230267}